## シンプルなLLMアプリケーション

### 準備

必要なライブラリのインストール

In [102]:
%pip install -r ../requirements.txt

  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
Note: you may need to restart the kernel to use updated packages.


環境変数の読み込み

In [1]:
from dotenv import load_dotenv
import os

load_dotenv("../.env")

True

### Chat Model

Cohereを利用した例

In [11]:
from langchain_cohere import ChatCohere

model = ChatCohere(
    model="command-r-plus",
    temperature=1,
)

from langchain_core.messages import HumanMessage

messages = [
    HumanMessage("こんにちは"),
]

model.invoke(messages)

AIMessage(content='こんにちは！今日はどうしましたか？', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '01c22606-d9a1-4dbb-8d43-047fd7ece229', 'token_count': {'input_tokens': 202.0, 'output_tokens': 12.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '01c22606-d9a1-4dbb-8d43-047fd7ece229', 'token_count': {'input_tokens': 202.0, 'output_tokens': 12.0}}, id='run-0351e511-65ce-41ef-bf5c-cdf39cf32d83-0', usage_metadata={'input_tokens': 202, 'output_tokens': 12, 'total_tokens': 214})

### Message

会話履歴を利用した例

In [12]:

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage("あなたはテンションの高い返信をするアシスタントです"),
    HumanMessage("僕はBigBaBy!BBと呼んでね"),
    AIMessage("BBだね。よろしく！"),
    HumanMessage("僕は誰ですか？"),
]

model.invoke(messages)

AIMessage(content='あなたはBigBaBy、BBです！覚えやすい名前だね。', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'f8a3d907-08a1-4f45-b0de-fb09f9cea370', 'token_count': {'input_tokens': 242.0, 'output_tokens': 16.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'f8a3d907-08a1-4f45-b0de-fb09f9cea370', 'token_count': {'input_tokens': 242.0, 'output_tokens': 16.0}}, id='run-f26d9976-37e7-437c-96a7-d7d9ccbf0d1a-0', usage_metadata={'input_tokens': 242, 'output_tokens': 16, 'total_tokens': 258})

nameプロパティを利用した話者の識別

In [5]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI

openai_model = ChatOpenAI(
  model="gpt-3.5-turbo",
  temperature=0.7,
  api_key=os.getenv("OPENAI_API_KEY"),
)

messages = [
    SystemMessage("あなたはテンションの高い返信をするアシスタントです"),
    HumanMessage(content="僕はBigBaBy!BBと呼んでね",name="BB"),
    AIMessage(content="BBだね。よろしく！"),
    HumanMessage(content="私はCC!シーツ―と呼んでね",name="CC"),
    HumanMessage(content="僕は誰ですか？",name="CC"),
]

openai_model.invoke(messages)

AIMessage(content='CCだよ！シーツ―！めっちゃかっこいい名前だね！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 100, 'total_tokens': 125, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a73eb9b5-8949-4a09-8c88-64c01f7332b3-0', usage_metadata={'input_tokens': 100, 'output_tokens': 25, 'total_tokens': 125})

### Prompt Template

#### String PromptTemplates

In [13]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("{topic}について教えて")

prompt_template.invoke({"topic": "BigBaBy"})

StringPromptValue(text='BigBaByについて教えて')

#### ChatPromptTemplates

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "あなたは有用なアシスタントです"),
    ("user", "{topic}について教えて")
])

prompt_template.invoke({"topic": "BigBaBy"})

ChatPromptValue(messages=[SystemMessage(content='あなたは有用なアシスタントです', additional_kwargs={}, response_metadata={}), HumanMessage(content='BigBaByについて教えて', additional_kwargs={}, response_metadata={})])

#### MessagesPlaceholder

In [93]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "あなたは有用なアシスタントです"),
    MessagesPlaceholder("msgs")
])

prompt_template.invoke({"msgs": [HumanMessage(content="こんにちは")]})

ChatPromptValue(messages=[SystemMessage(content='あなたは有用なアシスタントです', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは', additional_kwargs={}, response_metadata={})])

### OutputParser

CommaSeparatedListOutputParser

In [97]:
from langchain_core.output_parsers import  CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [99]:
parser.invoke("aaa,bbb,cc")

['aaa', 'bbb', 'cc']

XMLOutputParser

In [8]:
from langchain_core.output_parsers import XMLOutputParser

parser = XMLOutputParser()
parser.get_format_instructions()

'The output should be formatted as a XML file.\n1. Output should conform to the tags below. \n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema. \n2. String "<foo>\n   <bar>\n   </foo>" is a badly-formatted instance.\n3. String "<foo>\n   <tag>\n   </tag>\n</foo>" is a badly-formatted instance.\n\nHere are the output tags:\n```\nNone\n```'

In [9]:
parser.invoke("""<?xml version="1.0" encoding="UTF-8"?>
<bookstore>
  <book>
    <title lang="en">The Catcher in the Rye</title>
    <author>J.D. Salinger</author>
    <year>1951</year>
    <price>6.99</price>
  </book>
  <book>
    <title lang="ja">ノルウェイの森</title>
    <author>村上 春樹</author>
    <year>1987</year>
    <price>8.99</price>
  </book>
</bookstore>
""")

{'bookstore': [{'book': [{'title': 'The Catcher in the Rye'},
    {'author': 'J.D. Salinger'},
    {'year': '1951'},
    {'price': '6.99'}]},
  {'book': [{'title': 'ノルウェイの森'},
    {'author': '村上 春樹'},
    {'year': '1987'},
    {'price': '8.99'}]}]}

### 名前と呼び名を教えてくれるアプリケーション

In [15]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate,HumanMessagePromptTemplate,MessagesPlaceholder
from pydantic import BaseModel, Field

# データ構造の定義
class Human(BaseModel):
    name: str = Field(description="その人の名前")
    nickname: str = Field(description="その人の呼び名")

parser = JsonOutputParser(pydantic_object=Human)

prompt = PromptTemplate(
    template="質問に答えてください.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

human_message_prompt = HumanMessagePromptTemplate(prompt=prompt)


prompt_template = ChatPromptTemplate.from_messages([
    ("system", "あなたは有能なアシスタントでおもしろい呼び名を付ける才能があります"),
    MessagesPlaceholder("msgs"),
    human_message_prompt,
])

chain = prompt_template |model | parser


In [16]:

messages_history =  [
    HumanMessage("おっす!僕はBigBaBy!BBと呼んでね"),
    AIMessage("分かりました。あなたのことをBBと呼びます"),
    ]
chain.invoke({"msgs": messages_history,"query":"僕は誰ですか？"})

{'name': 'BigBaBy', 'nickname': 'BB'}

In [17]:
messages_history =  []
chain.invoke({"msgs": messages_history,"query":"総理大臣は誰ですか？"})

{'name': '岸田 文雄', 'nickname': 'キッシー'}